## Name: Krina Bharatbhai Patel
## Net Id : vj3004

## Name: Fnu,Sharmila Bolikoppa Palakshappa
## Net Id : bm6575

## Name: Vatsal Sudhir Pathak
## Net Id : pv2556

## Group Id: 7
## Assignment 2

## Q2 [3pts]: Use all the 64 features, assume that inputs are 64 dimensional Gaussians, and assume that for each class the covariance matrix is different. Report the training and test confusion matrices and errors. 



In [6]:
import numpy as np
from pandas import DataFrame, read_csv, to_numeric
from sklearn import cluster
import pandas as pd
from numpy.linalg import multi_dot
from sklearn.metrics import confusion_matrix


df_train = read_csv('optdigits_69_N200.tra',header=None)
df_test = read_csv('optdigits_69.tes',header=None)


In [7]:
# computing variance for each columns for each class
# if variance is below or equal to 0(var_threshold) for any column for any class, that column will be dropped

var_threshold = 0
df_var = df_train.groupby(by=64).var()
s_var_threshold = (df_var > var_threshold).product()
columns_to_drop = s_var_threshold[s_var_threshold == 0].index
print(columns_to_drop)

Int64Index([0, 1, 6, 7, 8, 14, 15, 16, 21, 22, 23, 24, 31, 32, 39, 40, 47, 48,
            56],
           dtype='int64')


In [8]:
# drop columns with variance below or equal to threshold
df_train = df_train.drop(columns = columns_to_drop)
df_test = df_test.drop(columns = columns_to_drop)

## Find statistics (mean, covariance matrix, inverse covariance matrix and probability) for each label (6,9)

In [9]:
grouped = df_train.groupby(by=64)
stat_list = []
for name, group in grouped:
    df = group.drop(columns=[64])
    m = np.mean(df)
    s = np.cov(df.T)
    s_inv = np.linalg.inv(s)
    prob = df.shape[0] / df_train.shape[0]
    stat_list.append([name, m, s, s_inv, prob])

df_stat = pd.DataFrame(stat_list, columns=['label', 'mean', 'cov', 'cov_inv', 'prob'])
df_stat = df_stat.set_index('label')
df_stat

,mean,cov,cov_inv,prob
label,,,,
6,2 1.62 3 10.39 4 8.17 5 1.0...,"[[8.601616161616176, 3.644646464646466, -7.813...","[[2.365819243333108, 0.10448503724247567, 0.17...",0.5
9,2 4.32 3 10.99 4 11.44 5 5.8...,"[[13.290505050505047, 12.98303030303031, -1.96...","[[1.0851332190027723, -0.2569954456264059, 0.2...",0.5


## Multivariate Discriminant Function

In [10]:
def discriminant_func(x, mean, cov, cov_inv, prob):    
    wi = -0.5 * cov_inv
    wj = np.dot(cov_inv, mean)
    wk = -0.5 * np.linalg.multi_dot([mean.T, cov_inv, mean]) - 0.5 * np.log(np.linalg.det(cov)) + np.log(prob)
    g = np.linalg.multi_dot([x.T, wi, x]) + np.dot(wj.T, x) + wk 
    return g.squeeze().astype('float')


## Multivariate Predict Function

In [11]:
def multivariate_predict(df, df_stat):
    df_no_label = df.drop(columns=[64])

    results = []
    for i in range(df_no_label.shape[0]):
        x = np.asarray(df_no_label.iloc[i])
        x = x.reshape(1,-1).T
        discriminant_values = df_stat.apply(lambda row:
                            discriminant_func(x, row["mean"], row["cov"], row["cov_inv"], row["prob"]), axis=1)

        discriminant_values = discriminant_values.astype('float')
        res = discriminant_values.idxmax()
        results.append(res)

    df_results = pd.DataFrame(results, columns=['predicted'])
    return df_results

## Find train and test errors

In [12]:
df_results_train = multivariate_predict(df_train, df_stat)
df_results_test = multivariate_predict(df_test, df_stat)

In [13]:
e = df_results_train.ne(df_train[64], axis=0).mean().sort_values()*100
print("Train error: %.4f %%"%(e.values[0]))

Train error: 0.0000 %


## Train error: 0.0000 %

In [14]:
e = df_results_test.ne(df_test[64], axis=0).mean().sort_values()*100
print("Test error: %.4f %%"%(e.values[0]))

Test error: 1.3850 %


## Test error: 1.3850 %

## Confusion Matrix
## 1) Confusion Matrix for Training

In [18]:
confusion_matrix(df_train[64], df_results_train)

array([[100,   0],
       [  0, 100]])

## 2) Confusion Matrix for testing

In [19]:
confusion_matrix(df_test[64], df_results_test)

array([[176,   5],
       [  0, 180]])